In [73]:
import torch
import pandas as pd
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [74]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [75]:
df.drop(columns=['id', 'Unnamed: 32'], inplace = True)

## Train Test Split

In [76]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:],df.iloc[:,0], test_size = 0.2)

## Scaling and Encoding

In [77]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

## Numpy to Pytorch

In [78]:
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

## Reshape y to match model output

In [79]:
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

# Creating Model

In [86]:
class Model(nn.Module):
  def __init__(self, input_features):
    super().__init__()
    self.network = nn.Sequential(
        nn.Linear(input_features, 15),
        nn.ReLU(),
        nn.Linear(15, 8),
        nn.ReLU(),
        nn.Linear(8, 4),
        nn.ReLU(),
        nn.Linear(4, 1),
        nn.Sigmoid()
    )


  def forward(self, X):
    a = self.network(X)
    return a


  def predict(self, X):
    with torch.no_grad():
      y = self.forward(X)
      y_pred = (y > 0.5).int()
      return y_pred

In [87]:
model = Model(30)

In [88]:
learning_rate = 0.1
epochs = 1000

In [89]:
loss_function = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [90]:
# Loop
for epoch in range(epochs):
  # Forward Pass
  y_pred = model(X_train)
  # Loss
  loss = loss_function(y_pred, y_train)
  #zeroing previous gradients
  optimizer.zero_grad()
  # Backward Pass
  loss.backward()
  # Update Gradients
  optimizer.step()
  if epoch==0 or epoch%100==99 or epoch==999:
    print(f'Epoch: {epoch+1}, Loss: {loss}')

Epoch: 1, Loss: 0.748353123664856
Epoch: 100, Loss: 0.0012049294309690595
Epoch: 200, Loss: 0.000341689185006544
Epoch: 300, Loss: 0.00014200642181094736
Epoch: 400, Loss: 7.391568215098232e-05
Epoch: 500, Loss: 4.562140748021193e-05
Epoch: 600, Loss: 3.0516543120029382e-05
Epoch: 700, Loss: 2.173335997213144e-05
Epoch: 800, Loss: 1.6390000382671133e-05
Epoch: 900, Loss: 1.253484242624836e-05
Epoch: 1000, Loss: 9.989915270125493e-06


In [91]:
y_pred_test = model.predict(X_test)

In [92]:
print(f'No of correct predictions: {torch.sum(y_pred_test==y_test)}')
print(f'No of incorrect predictions: {torch.sum(y_pred_test!=y_test)}')

No of correct predictions: 113
No of incorrect predictions: 1
